In [1]:
import json
import logging
from re import sub
from multiprocessing import cpu_count

import numpy as np
import pandas as pd 

import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

import nltk
from nltk.corpus import wordnet as wn

from gensim.models import Word2Vec

# pip install pymed

# nltk and pywsd # Libraries used 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer

import re, math
from re import sub

In [2]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.50d.txt'
word2vec_output_file = 'glove.6B.50d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

(400000, 50)

In [3]:
from gensim.models import KeyedVectors
# load the Stanford GloVe model
filename = 'glove.6B.50d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [4]:
len(model.vocab)

400000

In [6]:
model.wv['hobbled']

<ipython-input-6-1d372b3f8332>:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  model.wv['hobbled']


array([ 0.18539  , -1.0622   ,  0.22921  , -0.35302  , -0.88711  ,
        0.41132  , -0.40191  ,  0.44901  ,  0.32082  ,  0.0080124,
        0.15708  , -0.13979  , -1.3545   ,  0.62559  , -0.065345 ,
        0.44234  , -0.10911  , -0.11219  ,  0.77597  ,  0.104    ,
       -0.73884  ,  0.34582  , -0.76351  , -0.59176  ,  0.20951  ,
       -0.32046  ,  1.0912   ,  0.75365  ,  0.27159  ,  0.40587  ,
        0.71808  ,  0.57379  ,  1.221    ,  0.14095  , -0.5253   ,
        0.62881  , -0.29711  ,  0.41611  ,  0.15397  , -0.42621  ,
       -1.096    , -0.016531 ,  0.035577 ,  0.35256  ,  0.11429  ,
       -0.12316  ,  0.022998 ,  0.0087143, -0.37406  , -0.61669  ],
      dtype=float32)

In [7]:
# Takes a paragraph and returns the signature.

#paragraph = str(obj['paragraph'])
def para_signature(paragraph, word):
    sent_sig = [] # Stores sentence signature with POS

    # 1. Tokenize and remove punctuation
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(paragraph)
    
    # For POS Tagging
    def find_pos(new_words):
        pt = nltk.pos_tag(new_words)
        pos1 = ""
        for i in pt:
            if i[0]== word:
                pos1 = (i[1])
            else:
                continue
        return pos1

    def add_pos(pos_value):
        # make a dictionary of possible tags (in progress)
        pos_tag = {'verb':['VB','VBD','VBG','VBN','VBP','VBZ'], 
                   'noun': ['NN','NNS','NNP'], 
                   'adjective': ['JJ','JJR','JJS'],
                   'adverb':['RB', 'RBR', 'RBS']} # "masculine noun",

        for ele in pos_tag:
            for i in pos_tag[ele]:
                if i == pos:
                    return ele
                else:
                    continue

    

    # 2. POS Tagging
    pos = find_pos(new_words)
    pos_to_list = [add_pos(pos)]

    # 3. Convert all to lower case
    words = [word.lower() for word in new_words]

    # 4. Filter out stop words
    stop_words = stopwords.words('english')
    words_rem = [w for w in words if not w in stop_words]
    #words_rem= pos_to_list + words_rem   #add POS label to the list
    #sent_sig.append(words_rem)
    
    # 4. Lemmatize the words
    lmtzr = WordNetLemmatizer()
    lemmatized = [lmtzr.lemmatize(word) for word in words_rem]
    lemmatized= pos_to_list + lemmatized   #add POS label to the list
    sent_sig = lemmatized

    
    return sent_sig

#para_sig = para_signature(paragraph,word)
#print(para_sig)

In [8]:
# Takes word meanings and returns its signature with POS.

def meaning_signatures(dict_data,lu_word):
    signatures = [] # Stores meaning signatures with POS
    tokenizer = nltk.RegexpTokenizer(r"\w+")

    for i in range(len(dict_data)):

        # 1. Tokenize and remove punctuation
        dict_meaning= tokenizer.tokenize(dict_data[i].get("meaning"))

        # 2. Convert all to lower case
        word_set = [word.lower() for word in dict_meaning]

        # 3. Filter out stop words
        stop_words = stopwords.words('english')
        signature = [w for w in word_set if not w in stop_words]

        # 4. Lemmatize the words
        lmtzr = WordNetLemmatizer()
        lemmatized = [lmtzr.lemmatize(word) for word in signature]
        lemmatized= [dict_data[i].get("fl")] + lemmatized #add POS label to the list
        #signatures.append(lemmatized)
        
        # 6. Remove the look up word if present in meaning
        lemmatized[:] = [x for x in lemmatized if lu_word not in x]
        # Add stemmed meanings to a list
        signatures.append(lemmatized)
        
        
    return (signatures)    

#meaning_sig = meaning_signatures(dict_data, word)
#meaning_sig

In [9]:
def extract_ids(dicdata):
    ids = []
    for i in dict_data:
        mea_id = i['id']
        ids.append(mea_id)
    return ids

def extract_meanings(dicdata):
    meaning_list = []
    for i in dict_data:
        meaning = i['meaning'] 
        meaning_list.append(meaning)
    return meaning_list

def extract_pos_meanings(dicdata):
    pos_meaning = []
    for i in dict_data:
        pos = i['fl']
        meaning = i['meaning'] 
        pos_mea = pos + ' '+ meaning # POS + meaning
        pos_meaning.append(pos_mea)
    return pos_meaning

def extract_isteacher(dicdata):
    isteacher = []
    for i in dict_data:
        is_tea = i['isTeacher']
        isteacher.append(is_tea)
    return isteacher

def extract_pos(dicdata):
    pos = []
    for i in dict_data:
        pos_fl = i['fl']
        pos.append(pos_fl)
    return pos

In [10]:
# 1. Input
# JSON file 
# Read file
jsonfile= open('Json_wsd/request29.json','r',encoding='utf8')
data= jsonfile.read()

# Parse and Extract word, paragraph, pos and meanings
obj = json.loads(data)

word = str(obj['word'])
print("\nLookup word: ",word)

paragraph = str(obj['paragraph'])
print("\nParagraph: ",paragraph)

dict_data = obj['dictionaryData']
print(dict_data)
ids = extract_ids(dict_data)
meanings = extract_meanings(dict_data)
print(meanings)

pos = extract_pos(dict_data)
#pos_meanings = extract_pos_meanings(dict_data)
#print("\n",pos_meanings)
isteacher = extract_isteacher(dict_data)



Lookup word:  swagger

Paragraph:  Mild-mannered, he walked with more hunch than swagger. In class, he preferred listening to speaking. Patrick never bullied anybody. He never cussed anybody out. He appeared to abide by a self-imposed code: Keep to yourself, don’t mess around, don’t get involved with other people’s trouble. But he was willing to break his own code for a cause: Once, he leapt between two girls to stop a fight, and, in the process, got slammed to the ground.
[{'id': 2743, 'word_id': 861, 'fl': 'noun', 'meaning': 'a way of walking or behaving that shows you have a lot of confidence', 'count': 2, 'isTeacher': 1, 'isUser': 0}, {'id': 2744, 'word_id': 861, 'fl': 'verb', 'meaning': 'to walk in a very confident way : to walk with a swagger', 'count': 0, 'isTeacher': None, 'isUser': 0}, {'id': 5962, 'word_id': 861, 'fl': 'verb', 'meaning': 'to conduct oneself in an arrogant or superciliously pompous manner; especially : to walk with an air of overbearing self-confidence', 'cou

In [11]:
para_sig = para_signature(paragraph,word)
meaning_sig = meaning_signatures(dict_data, word)

vocab_w2v = list(model.wv.vocab) # list of all vocab
#vocab_w2v_wiki = list(word_vectors.wv.vocab) # list of all vocab
para_sig_r = [w for w in para_sig if w in vocab_w2v]

meaning_sig_r =[]
for i in meaning_sig:
    m_sig_r = [w for w in i if w in vocab_w2v]
    meaning_sig_r.append(m_sig_r)

# Find cosine similarity    
cos_sim = []
for i in meaning_sig_r:
    similarity =model.n_similarity(para_sig_r, i)
    cos_sim.append(similarity)
print(cos_sim)

zipped = list(zip(ids, meanings, cos_sim, isteacher))
features = sorted(zipped, key = lambda x: x[2], reverse = True)


ranked_meanings = pd.DataFrame(features, columns = ['id', 'meaning','score','isTeacher'])
ranked_meanings['isTeacher'] = ranked_meanings['isTeacher'].fillna(0)
ranked_meanings

[0.9302474, 0.8632548, 0.8343315, 0.9214905, 0.37468916, 0.8511383]


<ipython-input-11-0cbaf3bd97f7>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vocab_w2v = list(model.wv.vocab) # list of all vocab


,id,meaning,score,isTeacher
0,2743,a way of walking or behaving that shows you ha...,0.930247,1.0
1,5963,an arrogantly self-confident way of walking : ...,0.921490,0.0
2,2744,to walk in a very confident way : to walk with...,0.863255,0.0
3,5965,a short light stick usually covered with leath...,0.851138,0.0
4,5962,to conduct oneself in an arrogant or supercili...,0.834332,0.0
5,5964,marked by elegance or showiness : posh,0.374689,0.0


In [12]:
# Load Data and store

#request1train1_revised.json
jsonfile= open('data_600.json','r',encoding='utf8') # removed word aimless
data= jsonfile.read() 

# Parse and Extract word, paragraph, pos and meanings
obj = json.loads(data)

data = obj['data'] #[:10]
print("Total words: ",len(data)) # Total # of words

vocab_w2v = list(model.wv.vocab) # list of all vocab

count=0
# Store accuracy and MRR in blank lists
acc_score=[]
mrr_score=[]

# Parse data and find best meaning
for each_word_data in data:
    word = each_word_data['word']
    paragraph = each_word_data['paragraph']
    dict_data = each_word_data['dictionaryData']
    for i in dict_data:
        ids = extract_ids(dict_data)
        meanings = extract_meanings(dict_data)
        pos_meanings = extract_pos_meanings(dict_data)
        isteacher = extract_isteacher(dict_data)
        
        
    # Get signatures    
    para_sig = para_signature(paragraph,word)
    meaning_sig = meaning_signatures(dict_data, word)
    
    # Add tokens of meaning if meaning list is less than 2 words
    meaning_sig1 =[]
    for i,sig in enumerate(meaning_sig):
        if len(sig)<2:
            tokenizer = nltk.RegexpTokenizer(r"\w+")
            dict_meaning= tokenizer.tokenize(dict_data[1].get("meaning"))
            sig = sig+ dict_meaning
            meaning_sig1.append(sig)
        else:
            meaning_sig1.append(sig)  

    
    # Remove words not in Vocab
    #vocab_w2v = list(model.wv.vocab) # list of all vocab
    para_sig_r = [w for w in para_sig if w in vocab_w2v]

    meaning_sig_r =[]
    for i in meaning_sig1:
        m_sig_r = [w for w in i if w in vocab_w2v]
        meaning_sig_r.append(m_sig_r)

    # Compute Cosine Similarity Measure between the query and the documents.   
    cos_sim = []
    for i in meaning_sig_r:
        similarity =model.n_similarity(para_sig_r, i)
        cos_sim.append(similarity)
    
    
    # Output the document similarity results
    zipped = list(zip(ids, meanings, cos_sim, isteacher))
    features = sorted(zipped, key = lambda x: x[2], reverse = True)

    print("\n\nLookup word: ",word)
    ranked_meanings = pd.DataFrame(features, columns = ['id', 'meaning','score','isTeacher'])
    ranked_meanings['isTeacher'] = ranked_meanings['isTeacher'].fillna(0)
    ranked_meanings['pred_rank'] = ranked_meanings['score'].rank(pct=False, ascending= False)
    print(ranked_meanings)
    
     
    count +=1
    print(count)
    
    # Find accuracy
    if ranked_meanings.iloc[0]['isTeacher'] ==1:
        acc_score.append(1)
    else:
        acc_score.append(0)
        
    # Find MRR score
    istea_index = ranked_meanings[ranked_meanings['isTeacher']==1].index[0]+1
    pred_rank =1/istea_index
    mrr_score.append(pred_rank)

# Return Accuracy and MRR 
#print(acc_score)
avg = sum(acc_score)/len(acc_score)
print("The accuracy score is ", round(avg,4))

mrr = np.sum(mrr_score)/ len(mrr_score)
print("The Mean Reciprocal Rank is ", round(mrr,4)) 

Total words:  600


Lookup word:  circumvent 
     id                                            meaning     score  \
0  3230  to avoid being stopped by (something, such as ...  0.810221   

   isTeacher  pred_rank  
0          1        1.0  
1


Lookup word:  addict
     id                                            meaning     score  \
0  3401  a person who is not able to stop taking drugs ...  0.867434   
1  5885                    to cause addiction in (someone)  0.724558   
2  5884  one exhibiting a compulsive, chronic, physiolo...  0.645161   

   isTeacher  pred_rank  
0        0.0        1.0  
1        0.0        2.0  
2        1.0        3.0  
2


<ipython-input-12-c1d568cbb0d7>:13: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vocab_w2v = list(model.wv.vocab) # list of all vocab




Lookup word:  frustrated
     id                                            meaning     score  \
0  5886  feeling, showing, or characterized by frustrat...  0.729738   
1  5887                   to balk or defeat in an endeavor  0.508191   

   isTeacher  pred_rank  
0        1.0        1.0  
1        0.0        2.0  
3


Lookup word:  manslaughter
     id                                            meaning     score  \
0  2554  the crime of killing a person without intendin...  0.834936   

   isTeacher  pred_rank  
0          1        1.0  
4


Lookup word:  bludgeoned
     id                                  meaning     score  isTeacher  \
0  1080  to hit (someone or something) very hard  0.854667        1.0   
1  5697                 to hit with heavy impact  0.754494        0.0   

   pred_rank  
0        1.0  
1        2.0  
5


Lookup word:  envisioned
     id                                            meaning     score  \
0  2341  to think of (something that you believe might 



Lookup word:  resentments
     id                                            meaning     score  \
0  5938  a feeling of indignant displeasure or persiste...  0.729503   

   isTeacher  pred_rank  
0          1        1.0  
24


Lookup word:  constitutes
     id                 meaning     score  isTeacher  pred_rank
0  5944  make up, form, compose  0.734435          1        1.0
25


Lookup word:  befuddled
     id                                         meaning     score  isTeacher  \
0  2502         unable to think clearly : very confused  0.788649        0.0   
1  5945  utterly confused or puzzled : deeply perplexed  0.501821        1.0   
2  5946   to muddle or stupefy with or as if with drink  0.388873        0.0   

   pred_rank  
0        1.0  
1        2.0  
2        3.0  
26


Lookup word:  damning
     id                                            meaning     score  \
0  3477                                 yourself or itself  0.951335   
1  3476  —used to show that you are

Lookup word:  swampy
     id                                            meaning     score  \
0  5997  consisting of, suggestive of, or resembling sw...  0.421984   

   isTeacher  pred_rank  
0          1        1.0  
46


Lookup word:  juke
     id                                            meaning     score  \
0  5999  a small inexpensive establishment for eating, ...  0.758522   
1  3229             a cheap bar or club that has a jukebox  0.651856   
2  3228  to make a false movement in order to deceive (...  0.609180   
3  5998           to fake out of position (as in football)  0.592658   

   isTeacher  pred_rank  
0        1.0        1.0  
1        1.0        2.0  
2        0.0        3.0  
3        0.0        4.0  
47


Lookup word:  marquees
     id                                            meaning     score  \
0  6001  a large tent set up for an outdoor party, rece...  0.819063   

   isTeacher  pred_rank  
0          1        1.0  
48


Lookup word:  loiterers
     id      

...

Lookup word:  trudged
     id                                            meaning    score  \
0  6785  to walk or march steadily and usually laboriously  0.71782   

   isTeacher  pred_rank  
0          1        1.0  
486


Lookup word:  flitted
     id                                            meaning    score  \
0  6786  to pass quickly or abruptly from one place or ...  0.76238   

   isTeacher  pred_rank  
0          1        1.0  
487


Lookup word:  marijuana
     id                                            meaning     score  \
0  6802  marijuana that is available only by prescripti...  0.764821   
1  6801     opposed to or restricting the use of marijuana  0.732525   
2  6800  the psychoactive dried resinous flower buds an...  0.622791   
3  3702  the dried leaves and flowers of the hemp plant...  0.529097   

   isTeacher  pred_rank  
0        0.0        1.0  
1        0.0        2.0  
2        0.0        3.0  
3        1.0        4.0  
488


Lookup word:  grimaced
     id   

     id                                            meaning     score  \
0  6690  simultaneous performance of action or utteranc...  0.903210   
1  4712                          identity in musical pitch  0.599896   

   isTeacher  pred_rank  
0          1        1.0  
1          1        2.0  
512


Lookup word:  limbo
     id                                            meaning     score  \
0  6755  If you say that someone or something is in lim...  0.852422   
1  6754  in a forgotten or ignored place, state, or sit...  0.811482   
2  6752  a dance or contest that involves bending over ...  0.702993   
3  6751  an abode of souls that are according to Roman ...  0.623611   
4  6753  a tree (Bursera simaruba of the family Bursera...  0.481031   

   isTeacher  pred_rank  
0        1.0        1.0  
1        0.0        2.0  
2        0.0        3.0  
3        0.0        4.0  
4        0.0        5.0  
513


Lookup word:  aggressive
     id                                            meaning 



Lookup word:  haphazard
     id                                            meaning     score  \
0  5721        marked by lack of plan, order, or direction  0.893698   
1  4528  marked by lack of plan, order, or direction : ...  0.891640   
2  5722                                             chance  0.663277   

   isTeacher  pred_rank  
0        0.0        1.0  
1        1.0        2.0  
2        0.0        3.0  
536


Lookup word:  spectators
     id                      meaning     score  isTeacher  pred_rank
0  6894  one who looks on or watches  0.853138          1        1.0
537


Lookup word:  commenced
     id                                            meaning     score  \
0  4749  to bring or come into activity, being, or oper...  0.856939   

   isTeacher  pred_rank  
0          1        1.0  
538


Lookup word:  slurred
     id                                            meaning     score  \
0  4822  to slide or slip over without due mention, con...  0.776597   
1  4823      



Lookup word:  crappy
     id                               meaning    score  isTeacher  pred_rank
0  6944  markedly inferior in quality : lousy  0.61061          1        1.0
566


Lookup word:  sacred
     id                                            meaning     score  \
0  6947  one that is often unreasonably immune from cri...  0.809928   
1  6945  dedicated or set apart for the service or wors...  0.796096   
2  6950  a baboon (Papio hamadryas) that has a reddish-...  0.647321   
3  6949                                     magic mushroom  0.593046   
4  6948                                             peepul  0.428039   
5  6946                                   hamadryas baboon  0.125905   

   isTeacher  pred_rank  
0        1.0        1.0  
1        0.0        2.0  
2        0.0        3.0  
3        0.0        4.0  
4        0.0        5.0  
5        0.0        6.0  
567


Lookup word:  contraband
     id                                            meaning     score  \
0  468



Lookup word:  reprimand
     id                                            meaning     score  \
0  2463  to speak in an angry and critical way to (some...  0.904281   
1  4897  to reprove severely and usually formally from ...  0.670653   
2  4896                         a severe or formal reproof  0.487478   

   isTeacher  pred_rank  
0        1.0        1.0  
1        0.0        2.0  
2        0.0        3.0  
598


Lookup word:  fiddles 
     id                                            meaning     score  \
0  7013  to move or handle (something) with the hands o...  0.944587   
1  7012  to spend time in activity that does not have a...  0.854585   
2  7011    one that plays a supporting or subservient role  0.756719   
3  7010                                    to fritter away  0.602351   
4  7007                                to play on a fiddle  0.576990   
5  7008                                        double bass  0.553027   
6  7009                                        d